# Project: Facial emotion recognition
# MTH767P - Group 4

In [72]:
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

## Data loading and pre-processing 

In [4]:
# TODO: 
# - code loading of the dataset (train and test) 
# - code viewing samples of dataset (and maybe some other visualisation of the data characteristics)
# - code pre-processing/transformations

---
## Convolutional neural network setup 
### Network architecture

In [69]:
class Net(nn.Module):

    def __init__(self, layers, activation, pooling=None):
        super().__init__()
        layer_list = []
        for layer in layers:
            layer_list.append(layer.pop('ltype')(**layer))
        self.layers = nn.ModuleList(layer_list)
        self.activation = activation
        self.pool = pooling

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            print(f'layer {i+1}: {layer}')
            if isinstance(self.layers[i-1], nn.Conv2d) and isinstance(layer, nn.Linear):
                print("flattening")
                x = torch.flatten(x, 1)
            x = layer(x)
            if i != len(self.layers)-1:
                print("activation")
                x = self.activation(x)
                if isinstance(layer, nn.Conv2d) and self.pool is not None:
                    print("pooling")
                    x = self.pool(x)
            print(x)
        return x

In [70]:
# Architecture
l1 = dict(ltype=nn.Conv2d, in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0, groups=1, bias=True, padding_mode='zeros')
l2 = dict(ltype=nn.Conv2d, in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, groups=1, bias=True, padding_mode='zeros')
l3 = dict(ltype=nn.Linear, in_features=400, out_features=120, bias=True)
l4 = dict(ltype=nn.Linear, in_features=120, out_features=84, bias=True)
l5 = dict(ltype=nn.Linear, in_features=84, out_features=10, bias=True)
pooling = nn.MaxPool2d(2)
activation = F.relu
arch = {"layers": [l1,l2,l3,l4,l5], "pooling": pooling, "activation": activation}
net = Net(**arch)

### Loss function and optimizer

In [73]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

---
## Training the neural network
### Training

In [ ]:
num_epochs = 2

for epoch in range(num_epochs): 

    running_loss = 0.0
    # FIXME: below line needs to be changed to actual data loader
    for i, data in enumerate(trainloader, 0): 
        # FIXME: make sure the data item does contain both inputs and labels
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:  
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

### Cross-validation

In [74]:
# TODO:
# - write cross-validation routine to fine tune hyperparameters

In [ ]:
# Save trained model
PATH = '.'
torch.save(net.state_dict(), PATH)

---
## Testing

In [ ]:
# Load previously trained model
net = Net(**arch)
net.load_state_dict(torch.load(PATH))

In [75]:
# TODO:
# - code prediction function
# - code routine to calculate performance metrics
# - code visualisation of results